In [1]:
import pandas as pd
import numpy as np
import mpu
import seaborn as sns
import re

In [ ]:
import geopandas as gpd
import geopy
from shapely.geometry import Point, Polygon
from geopy.geocoders import Nominatim

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [4]:
geo_df = gpd.read_file('Reading shape files/egy_admbnda_adm3_capmas_20170421.shp')
geo_df.head()

,ADM3_EN,ADM3_AR,ADM3_PCODE,ADM3_REF,ADM3ALT1EN,ADM3ALT2EN,ADM3ALT1AR,ADM3ALT2AR,ADM2_EN,ADM2_AR,ADM2_PCODE,ADM1_EN,ADM1_AR,ADM1_PCODE,ADM0_EN,ADM0_AR,ADM0_PCODE,date,validOn,validTo,Shape_Leng,Shape_Area,geometry
0,None,الجزيره,EG120918,None,None,None,None,None,Dikirnis,مركز دكرنس,EG1209,Dakahlia,الدقهلية,EG12,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,0.055274,0.000157,"POLYGON ((31.54448 31.08392, 31.54505 31.07872..."
1,None,الجهاد 4,EG240525,None,None,None,None,None,Markz Al Idwa,مركز العدوة,EG2405,Menia,المنيا\n,EG24,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,0.101889,0.000594,"POLYGON ((30.73757 28.74247, 30.73682 28.74252..."
2,None,أم الرضا الجديدة,EG110437,None,None,None,None,None,Kafr Sad,مركز كفر سعد,EG1104,Damietta,دمياط\n,EG11,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,0.049183,0.000103,"POLYGON ((31.63090 31.41011, 31.62543 31.40981..."
3,None,رأس محمد,EG350502,None,None,None,None,None,Sharm el-Sheikh,قسم شرم الشيخ,EG3505,South Sinai,جنوب سيناء,EG35,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,0.044184,0.000094,"POLYGON ((34.24396 27.75904, 34.24485 27.74441..."
4,None,مبارك,EG120565,None,None,None,None,None,Sinbillawin,مركز السنبلاوين,EG1205,Dakahlia,الدقهلية,EG12,Egypt,مِصر,EG,2006-01-01,2017-04-21,None,0.044160,0.000122,"POLYGON ((31.36003 30.86623, 31.36525 30.86547..."


In [5]:
shapefile = geo_df[['ADM1_AR','ADM2_AR','ADM3_AR','geometry']]
shapefile.head()

,ADM1_AR,ADM2_AR,ADM3_AR,geometry
0,الدقهلية,مركز دكرنس,الجزيره,"POLYGON ((31.54448 31.08392, 31.54505 31.07872..."
1,المنيا\n,مركز العدوة,الجهاد 4,"POLYGON ((30.73757 28.74247, 30.73682 28.74252..."
2,دمياط\n,مركز كفر سعد,أم الرضا الجديدة,"POLYGON ((31.63090 31.41011, 31.62543 31.40981..."
3,جنوب سيناء,قسم شرم الشيخ,رأس محمد,"POLYGON ((34.24396 27.75904, 34.24485 27.74441..."
4,الدقهلية,مركز السنبلاوين,مبارك,"POLYGON ((31.36003 30.86623, 31.36525 30.86547..."


In [6]:
shapefile.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5716 entries, 0 to 5715
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   ADM1_AR   5716 non-null   object  
 1   ADM2_AR   5716 non-null   object  
 2   ADM3_AR   5716 non-null   object  
 3   geometry  5716 non-null   geometry
dtypes: geometry(1), object(3)
memory usage: 178.8+ KB


In [7]:
shapefile['ADM1_AR'] = shapefile['ADM1_AR'].apply(lambda x : re.sub('\n','',x))
shapefile['ADM2_AR'] = shapefile['ADM2_AR'].apply(lambda x : re.sub('\n','',x))
shapefile['ADM3_AR'] = shapefile['ADM3_AR'].apply(lambda x : re.sub('\n','',x))


/home/abduallah/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
shapes = shapefile['geometry']
type(shapes)

geopandas.geoseries.GeoSeries

In [9]:
atm_df = pd.read_excel('ATMS_bank_misr(add,lat,long).xlsx')
atm_df.head(5)

,Address,Lat,Long
0,Alfayoum Beghous Alhoriea St.,29.304412,30.853546
1,Fayoum University,29.319696,30.835641
2,Fayoum AlHoriea St.,29.308199,30.842229
3,Sanoras - Fayoum,29.408575,30.865073
4,194 Abo Bakr AlSiddiq St.- Atsa City - Fayoum,29.236966,30.790194


In [10]:
shapes.shape

(5716,)

In [14]:
def getpolygon (lat,lon):
    # transform lat and long into a point obj
    point = Point(lon, lat)
    
    distances = []
    # get distances between the point and all polygons so it is 1 * 5716 operation
    for i in range (len(shapes)):
        distances.append(point.distance(shapes[i]))
    # return the index of the minimum distance
    # the index is respect to the shapefile which contain all sub regions
    return np.array(distances).argmin()


In [15]:
# test the getpolygon function with some lat and long
print(getpolygon(28.10486134,30.7515505))
shapefile.iloc[getpolygon(28.10486134,30.7515505)]

3966


ADM1_AR                                                المنيا
ADM2_AR                                            قسم المنيا
ADM3_AR                                              قسم ثالث
geometry    POLYGON ((30.75615195100005 28.10614226100006,...
Name: 3966, dtype: object

In [18]:
def get_places(input_table):
    # get the index of the minimum distance in the shapefile
    index = getpolygon(input_table[0],input_table[1])

    location = shapefile.iloc[index]
        
    gov = location['ADM1_AR']
    region = location['ADM2_AR']
    subregion = location['ADM3_AR']
                
    # map every feature into it's column
    input_table['محافظة'] = gov
    input_table['قسم / مركز'] = region
    input_table['منطقة'] = subregion
    
    return input_table

In [ ]:
# apply the function, the complexity is 5716 * 5716 so it may take about 10 or 15 min to execute
atm_df = atm_df[['Lat','Long','Address']].apply(get_places, axis = 1)
atm_df

In [ ]:
# insert id column 
atm_df.insert(0,'ID',range(1,len(atm_df)+1))
atm_df

In [ ]:
# change columns' positions
atm_df = atm_df.loc[:,['ID','Address','منطقة','قسم / مركز','محافظة','Lat','Long']]

In [ ]:
atm_df.info()

In [ ]:
atm_df

In [ ]:
atm_df.to_excel('Atms_bankmisr_AR.xlsx',index= False)

In [ ]:
# loading the file so we don't have to execute the cells above again
# atm_df = pd.read_excel('Atms_bankmisr_AR.xlsx')
# atm_df.head()

In [22]:
branches_df = pd.read_excel('Branches_Location.xlsx')
branches_df

,Branch_Address,Lat,Long
0,Banque Misr Damanhour Branch - 1 Saad Zaghloul...,31.042419,30.468909
1,Albahr St. - Rasheed - Beside the school compl...,31.406311,30.423861
2,Mahmoudia Canal St.,31.182751,30.527557
3,Mohamed Metwally Alshaarawy St. - Abo Almatamir,30.910789,30.177019
4,Kafr Aldawar - Port Said St.,31.133868,30.135130
...,...,...,...
730,Arrival hall Suez Port,29.942901,32.567302
731,"Port Tawfiq in front of Gate No. (5), the cust...",29.942304,32.560966
732,Arrival visa office at the new airport adminis...,29.937393,32.557254
733,Arkan Mall - Sheikh Zayed Entrance 2 - in fron...,29.964206,32.549570


In [23]:
branches_df = branches_df[['Lat','Long','Branch_Address']].apply(get_places,axis =1)

In [24]:
branches_df.to_excel('Branches_Location.xlsx',index=0)

In [25]:
branches_df.head()

,Lat,Long,Branch_Address,محافظة,قسم / مركز,منطقة
0,31.042419,30.468909,Banque Misr Damanhour Branch - 1 Saad Zaghloul...,البحيرة,قسم دمنهور,طاموس
1,31.406311,30.423861,Albahr St. - Rasheed - Beside the school compl...,البحيرة,مركز رشيد,مدينة رشيد
2,31.182751,30.527557,Mahmoudia Canal St.,البحيرة,مركز المحمودية,كفر أمليط
3,30.910789,30.177019,Mohamed Metwally Alshaarawy St. - Abo Almatamir,البحيرة,مركز ابوالمطامير,مدينة أبو المطامير
4,31.133868,30.135130,Kafr Aldawar - Port Said St.,البحيرة,قسم كفرالدوار,مدينة كفر الدوار
